# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [1]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [2]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [3]:
describe(boston)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Nothing,Nothing,DataType
1,Crim,3.61352,0.00632,0.25651,88.9762,,,Float64
2,Zn,11.3636,0.0,0.0,100.0,,,Float64
3,Indus,11.1368,0.46,9.69,27.74,,,Float64
4,Chas,0.06917,0,0.0,1,,,Int64
5,NOx,0.554695,0.385,0.538,0.871,,,Float64
6,Rm,6.28463,3.561,6.2085,8.78,,,Float64
7,Age,68.5749,2.9,77.5,100.0,,,Float64
8,Dis,3.79504,1.1296,3.20745,12.1265,,,Float64
9,Rad,9.54941,1,5.0,24,,,Int64


In [4]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8)

([49, 131, 175, 169, 109, 314, 210, 180, 432, 485  …  119, 58, 246, 156, 166, 19, 282, 342, 202, 305], [272, 251, 235, 409, 506, 366, 392, 380, 40, 238  …  135, 195, 407, 92, 448, 438, 225, 362, 236, 170])

In [5]:
train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.16211,20.0,6.96,0,0.464,6.24,16.3,4.429,3,223
2,0.1403,22.0,5.86,0,0.431,6.487,13.0,7.3967,7,330
3,0.44791,0.0,6.2,1,0.507,6.726,66.5,3.6519,8,307
4,7.40389,0.0,18.1,0,0.597,5.617,97.9,1.4547,24,666
5,0.04741,0.0,11.93,0,0.573,6.03,80.8,2.505,1,273
6,4.55587,0.0,18.1,0,0.718,3.561,87.9,1.6132,24,666
7,5.29305,0.0,18.1,0,0.7,6.051,82.5,2.1678,24,666
8,17.8667,0.0,18.1,0,0.671,6.223,100.0,1.3861,24,666
9,0.02763,75.0,2.95,0,0.428,6.595,21.8,5.4011,3,252


In [6]:
ols = GLM.lm(@formula(MedV ~ Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

MedV ~ 1 + Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat

Coefficients:
─────────────────────────────────────────────────────────────────────────────────────
                 Estimate  Std. Error    t value  Pr(>|t|)     Lower 95%    Upper 95%
─────────────────────────────────────────────────────────────────────────────────────
(Intercept)   38.8415      5.77008      6.73154     <1e-10   27.4972      50.1858
Crim          -0.109978    0.0382273   -2.87694     0.0042   -0.185134    -0.0348208
Zn             0.0615879   0.0154155    3.99518     <1e-4     0.0312802    0.0918956
Indus          0.016847    0.0728532    0.231246    0.8172   -0.126386     0.16008
Chas           2.30515     0.944612     2.44031     0.0151    0.447993     4.1623
NOx          -17.9755      4.22133     -4.2

In [7]:
predict(ols, test)

101-element Array{Union{Missing, Float64},1}:
 27.86799452362333
 24.660311647473925
 31.149491302880534
 13.902214224647611
 22.639549136023327
 15.857866545362478
 17.473281260165834
 16.794660622246273
 32.614769898817165
 32.24927997943323
 11.830376624721431
 28.413712204694804
 16.985838236240266
  ⋮
 20.459464904415903
 17.07049865690348
 13.341832330369447
 32.245527421105315
  8.941668952215485
 27.304042933945027
 18.155919311494145
  8.793741959587704
 37.57481769983744
 19.05931180722996
 25.328134709187662
 26.0417358785166

In [8]:
GLM.r²(ols)

0.7243074054101197

In [9]:
GLM.adjr²(ols)

0.7151411554621185